<a href="https://colab.research.google.com/github/hyelim-kim1028/Algorithm_Study/blob/main/SQLite3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SQLite3 API 정리 
# SQL은 DB based 이고, SQLite은 file based 

# show lines in the cell shortcut: ctrl + M + L 

import sqlite3
con = sqlite3.connect('example.db')
# connection 객체: 데이터베이스 나타냄 
# 데이터는 example.db에 저장됨 

In [ ]:
cur = con.cursor() # cursor객체 생성 

# execute() 메서드 호출 > SQL 수행 

# Create table 
cur.execute('''CREATE TABLE stocks
               (date text, trans text, symbol text, qty real, price real)''')
# 같은 코드가 두번 돌면 > already exits 에러가 뜬다 


#insert a row of data 
cur.execute("INSERT INTO stocks VALUES('2006-01-05','BUY','RHAT',100,35.14)")

# save (commit) the chantes 
con.commit()

# close the connection 
con.close()

In [ ]:
# 데이터를 영구 저장 후 다시 사용 가능 
con = sqlite3.connect('example.db')
cur = con.cursor()

In [ ]:
cur.execute("INSERT INTO stocks VALUES('2006-03-28', 'BUY', 'IBM', 1000, 45.0)")

In [ ]:
cur.execute("INSERT INTO stocks VALUES('2006-04-06', 'SELL', 'IBM', 500, 53.0)")
cur.execute("INSERT INTO stocks VALUES('2006-04-05', 'BUY', 'MSFT', 1000, 72.0)")

In [ ]:
# iterator 사용 
# SELECT 문을 실행한 후 데이터를 꺼내려면, 
# 1) 커서를 이터레이터로 취급하거나, 
# 2)커서의 fetchone() 메서드를 호출하여 일치하는 단일 행을 꺼내거나,
# 3)  fetchall()를 호출하여 일치하는 행의 리스트를 가져올 수 있습니다.

In [ ]:
# 1) iterator 사용 방식 

for row in cur.execute('SELECT * FROM stocks ORDER BY Price'): 
  print(row)


('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)


In [ ]:
# Never do this -- insecure! 
# Do not use python's string operations 
symbol = 'RHAT'
cur.execute("SELECT * FROM stocks where symbol = '%s  % symbol")

In [ ]:
# 1. Put a placeholder wherever you want to use a value 
# placeholder: a. question mark b. named style > sequence or dict instance 
# the length of the sequence must be met > if not ProgrammingError 
# 2. provide a tuple of values as the second argument to the cursor's execute() method 

import sqlite3 

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("create table lang (name, first_appeared)")

# This is the qmark style: 
cur.execute("Insert into lang values (?,?)", ("C",1972))

# The qmark style used with executemany():
lang_list = [
             ("Fortran", 1957),
             ("Python", 1991),
             ("Go", 2009)
]

cur.executemany("insert into lang values (?,?)", lang_list)

# And this is the named style: 
cur.execute("select * from lang where first_appeared =:year ", {"year":1972})
print(cur.fetchall())

con.close()

[('C', 1972)]


In [ ]:
# SQLite documentation: https://www.sqlite.org/index.html
# PEP 249 데이터베이스 API 명세 2.0 : https://www.python.org/dev/peps/pep-0249/

In [ ]:
# 모듈 함수와 상수 

sqlite3.version
sqlite3.version_info
sqlite3.PARSE_DECLTYPES  # connect의 detect_types의 매개변수 
sqlite3.connect
# sqlite3.connect(database[, timeout, detect_types, isolation_level, check_same_thread, factory, cached_statements, url])
# databse에 연결 
# database : 파일의 경로명 / ":memory:"

# datatypes: text, integer, real, blob, null 
# real:  Real values are real numbers with decimal values that use 8-byte floats.
# blob : BLOB stands for a binary large object that can store any kind of data. The maximum size of BLOB is, theoretically, unlimited.

# detect types 
# default = 0 

# check_same_thread = True (dfault)
# True: 만들고 있는 스레드만이 연결을 사용할 수 있음 
# False: 연결을 여러 스레드에서 공유 가능 # 데이터 손상 피하기 >직렬화 
# what is 직렬화? 



<function _sqlite3.connect>

기본적으로, sqlite3 모듈은 connect 호출에 Connection 클래스를 사용합니다. 그러나, Connection 클래스의 서브 클래스를 만들고 factory 매개 변수에 클래스를 제공하면 connect()가 그 클래스를 사용하게 할 수 있습니다 (눼? 선생님...?) 

reference: https://docs.python.org/ko/3/library/sqlite3.html#sqlite3-types

In [ ]:
db = sqlite3.connect('file:path/to/database?mode=ro', uri=True)
# uri = True, 옵션 지정 가능 i.e. 읽기 전용 모드 
# https://www.sqlite.org/uri.html 

In [ ]:
sqlite3.register_adapter(type, callable)
# python type > SQLite convertable callable 
# int, float, str or bytes 

sqlite3.complete_statemet(sql)
# only checks if exists a semi-colon (not if gramatically correct)

# isolation_level 
# deferred, immediate, exclusive or None (자동 커밋 모드)

sqlite3.enable_callback_tracebacks(flags)


In [1]:
# A minimal SQLite shell for experiments 

import sqlite3 

con = sqlite3.connect(":memory:")
con.isolation_level = None 
cur = con.cursor() 

buffer = ""

print("Enter your SQL commands to execute in sqlite3")
print("Enter a blank line to exit.")

while True: 
  line = input()
  if line == "":
    break 
  buffer += line 
  if sqlite3.complete_statement(buffer): 
    try: 
      buffer = buffer.strip()
      cur.execute(buffer)

      if buffer.istrip().upper().startswith("SELECT"):
        print(cur.fetchall())
    except sqlite3.Error as e: 
      print("An error occurred:", e.args[0])
    buffer = ""

con.close()

# enable_callback_tracebacks(flag)
# to debug, flag = True > get tracebacks from callbacks on sys.stderr 
# flag = False > not get any tracebacks from user-defined functions, aggregates, etc 

Enter your SQL commands to execute in sqlite3
Enter a blank line to exit.
yes
pardoon
well



In [ ]:
# Connection 객체 
# class sqlite3.Connection 

# isolation_level 
# autocommit mode 

# in_transaction 
# true : transaction active 
# false: otherwise, read-only attribute 

# cursor (factory = Cursor)
# must be a callable returning an instance of Cursor 

# commit()
# commits the current transaction 

# rollback()
# back to db since the last call to commit()

# close()
# close the db connection 
# not automatically commit()

# execute(sql [,parameters])
# calls cursor() > execute() + parameters > returns the cursor

# executemany(sql [,parameters])

# executescript(sql_script)
# calls cursor 
# execturescript with sql_script 
# returns cursor()

# create_function (name, num_params, func, *, deterministic = False)
# 
